In [ ]:
pip install -qqq transformers datasets pandas cohere datasets altair topically umap-learn bertopic openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
# great full explination of this code: https://www.pinecone.io/learn/bertopic/

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Download the all_embeddings.npy file, and upload it to your own google drive (in the base folder, to make this demo work)
# https://drive.google.com/file/d/1xPR7a25AdhdN1B4yls1h066ccGgm8r9D/view?usp=share_link

# embeddings are pre-computed from Cohere API.

import numpy as np
all_embeddings = np.load('drive/MyDrive/content/all_embeddings.npy')
print(all_embeddings.shape)

(10000, 4096)


In [ ]:
## To install required dependencies, run the following command in the command line:
# pip install pandas cohere datasets altair topically umap-learn bertopic

# Import required libraries
import pandas as pd
import numpy as np
import cohere
import umap
import altair as alt
from bertopic import BERTopic
from datasets import load_dataset
from typing import Optional, List
import time

# Get (a small sample) the dataset
# dataset = load_dataset("AmazonScience/massive", "en-US", split="train")
df = pd.read_csv('drive/MyDrive/content/df_10k_cleaned.csv')
# df = pd.DataFrame(dataset)#.sample(300)

# filter out stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

df['text'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
del df['content']

In [ ]:
df

,date_posted,title,date,text
0,2011-06-08T14:22:00Z,Re: MOST PEOPLE dont Date or marry INTERRACIALY!,2011-06-08,"Well, must say I'm pleasantly surprised low pe..."
1,2009-09-06T07:03:00Z,Re: What Africa does not prove,2009-09-06,Remember San bows arrows. miss said that? Negr...
2,2005-05-16T18:20:00Z,Help I am unable to post,2005-05-16,unable post!! must :( 2508
3,2008-08-28T15:05:00Z,Re: WN is fantasy at best,2008-08-28,"Seems me, white male, fact real hope sort WN s..."
4,2008-09-02T13:22:00Z,Re: WN is fantasy at best,2008-09-02,"Ironically, presence indoctrinated beta males ..."
...,...,...,...,...
9995,2006-07-25T06:25:00Z,Re: Finland deports criminal Somalis back to S...,2006-07-25,really programmes Finland???:mad: believed ext...
9996,2006-07-25T19:41:00Z,Re: Finland deports criminal Somalis back to S...,2006-07-25,think one real nationalist Finn pay TV-licence...
9997,2006-07-25T20:15:00Z,Re: Finland deports criminal Somalis back to S...,2006-07-25,would easy change. problems are. Trouble peopl...
9998,2006-09-07T08:22:00Z,Re: Finland deports criminal Somalis back to S...,2006-09-07,cool thing Finland law assaulting someone rega...


In [ ]:
# from sklearn.cluster import KMeans
# n_clusters = 20
# cluster_model = KMeans(n_clusters=n_clusters)
# topic_model = BERTopic(hdbscan_model=cluster_model)

import openai
from bertopic.representation import OpenAI
from bertopic import BERTopic

openai.api_key = 'YOUR API KEY'
representation_model = OpenAI()

topic_model = BERTopic(representation_model=representation_model)

# df is a dataframe. df['title'] is the column of text we're modeling
df['topic'], probabilities = topic_model.fit_transform(df['text'], all_embeddings)

# generate cluster keywords
keywords = topic_model.generate_topic_labels()
df['cluster_keywords'] = df['topic'].map(lambda x: keywords[x])
df.head(10)

,date_posted,title,date,text,topic,cluster_keywords
0,2011-06-08T14:22:00Z,Re: MOST PEOPLE dont Date or marry INTERRACIALY!,2011-06-08,"Well, must say I'm pleasantly surprised low pe...",9,8_Chinese borrowings of ideas__
1,2009-09-06T07:03:00Z,Re: What Africa does not prove,2009-09-06,Remember San bows arrows. miss said that? Negr...,65,64_Environmental impacts of eating meat__
2,2005-05-16T18:20:00Z,Help I am unable to post,2005-05-16,unable post!! must :( 2508,106,105_WHITE AMERICANS in Europe__
3,2008-08-28T15:05:00Z,Re: WN is fantasy at best,2008-08-28,"Seems me, white male, fact real hope sort WN s...",-1,158_Health care in certain areas__
4,2008-09-02T13:22:00Z,Re: WN is fantasy at best,2008-09-02,"Ironically, presence indoctrinated beta males ...",146,145_Environmental impacts of playing video gam...
5,2006-02-24T08:48:00Z,"Re: New Members From Around The World, Post Yo...",2006-02-24,Hello everyone! Livewire :goof,-1,158_Health care in certain areas__
6,2008-07-08T12:51:00Z,Re: Black S.T.D. Statistics,2008-07-08,"several levels actually. First, educate others...",43,42_Risks and potential avoidance__
7,2008-07-08T17:15:00Z,Re: Black S.T.D. Statistics,2008-07-08,"Listen bonehead, want know thread started, ask...",9,8_Chinese borrowings of ideas__
8,2008-07-12T04:52:00Z,Re: THE MUGABE YOU DIDN'T KNOW,2008-07-12,Because... know business Iran. agree Africa BL...,5,4_PG-13 ratings__
9,2006-12-10T08:02:00Z,Re: Antis whats your problem?,2006-12-10,"Theoretically,you wrong.The 14th amendment Con...",117,116_Social effects of eating meat__


In [ ]:
def interactive_clusters_scatterplot(
        df: pd.DataFrame,
        fields_in_tooltip: List[str] = None,
        title: str = '',
        title_column: str = 'keywords'
):
    if fields_in_tooltip is None:
        fields_in_tooltip = ['']

    selection = alt.selection_multi(fields=[title_column], bind='legend')

    chart = alt.Chart(df).transform_calculate(
    ).mark_circle(size=80, stroke='#666', strokeWidth=1, opacity=0.1).encode(
        x=
        alt.X('x',
              scale=alt.Scale(zero=False),
              axis=alt.Axis(labels=False, ticks=False, domain=False)
              ),
        y=
        alt.Y('y',
              scale=alt.Scale(zero=False),
              axis=alt.Axis(labels=False, ticks=False, domain=False)
              ),

        color=alt.Color(f'{title_column}:N',
                        legend=alt.Legend(columns=2,
                                          symbolLimit=0,
                                          orient='right',
                                          labelFontSize=12)
                        ),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
        tooltip=fields_in_tooltip
    ).properties(
        width=800,
        height=700
    ).add_selection(
        selection
    ).configure_legend(labelLimit=0).configure_view(
        strokeWidth=0
    ).configure(background="#F6f6f6").properties(
        title=title
    ).configure_range(
        category={'scheme': 'category20'}
    )
    return chart

# Reduce dimensions to be able to plot the embeddings
n_neighbors = 15
reducer = umap.UMAP(n_neighbors=n_neighbors)
umap_embeds = reducer.fit_transform(all_embeddings)
df['x'] = umap_embeds[:, 0]
df['y'] = umap_embeds[:, 1]

# Specify the names of columns to plot

title_column = 'cluster_keywords'
fields_in_tooltip = ['text',  'topic', 'cluster_keywords']

title = "Topics in Stormfront (small 10k sample)"

# allow over 5k rows to be displayed
alt.data_transformers.disable_max_rows()

# chart = interactive_clusters_scatterplot(df,
#                                             fields_in_tooltip=fields_in_tooltip,
#                                             title=title + " - " + str(n_clusters) + " clusters",
#                                             title_column=title_column)
chart = interactive_clusters_scatterplot(df,
                                            fields_in_tooltip=fields_in_tooltip,
                                            title=title,
                                            title_column=title_column)
chart

In [ ]:
# print all topics and their ID numbers
for topic_num, topic_name in topic_model.get_topics().items(): print(topic_num, topic_name[0][0])

# I don't know why there are 148 topics. I think there's a default 'KNN distance threshold' causing this.

-1 useless information
0 ledgering
1 Zionism
2 gun control
3 Republican Party Convention
4 PG-13 ratings
5 Environmental implications of eating meat
6 Environmental impacts of eating meat
7 Environmental Impacts of Eating Meat
8 Chinese borrowings of ideas
9 Roles of white women in Tulara Valley
10 racism in the white community
11 ç
12 uncertain futures
13 Environmental shocks experienced when trying to leave the scene
14 Environmental effects of eating meat
15 Genetic variations inside races
16 Alpha males and their Pack
17 iffs:Gas Chambers and the Gas-Bomb- shelters
18 Free speech
19 Environmental impacts of eating meat
20 Environmental impacts of eating meat
21 Climate change
22 Linguistic impact of eating meat
23 Board quality andTextOS
24 Ethnicity
25 impossibility of boycotting pro sports
26 Environmental Dickens
27 Unemployment in China
28 Impact of eating meat on environmental impacts
29 White power people
30 uzzle
31 Transport issues
32 Environmental impacts of eating meat
33

In [ ]:
# You can get 'representative' documents from each category. Just pass in the category ID.

# get the ID number from the visualization above.
topic_model.get_representative_docs(8)

["(Woe moderator read this. read it, hope don't, sympathies.) said large, minute exception edges, different races develop independantly, almost total isoaltion other, vast majority history. Nobody denies minute exception edges, is, minute exception edges. minute exception edges stuff pass. significance impact interactions often quite momentous. legitimate history professor would take seriously told racial/cultural interaction insignificant. (Some examples scattered throughout.) people traded people races, probably way less 0.0001%,[...] Source? Classical era (500 BCE 500 CE), development long distance trade routes (the Silk Road included), revivals Post-Classical era (500 CE 1000 CE), definitely sometimes referred Age Cross-Cultural Interaction (1000 CE 1500 CE), case, definitely case civilizations continued grow. anything, 0.0001% measure relation who-knows-what. [...]wars people races involved people, sporadic history,[...] course, talking cultures, too, races. Needless say, even rac

In [ ]:
topic_model.get_document_info(df['text'])

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,"Well, must say I'm pleasantly surprised low pe...",9,9_Roles of white women in Tulara Valley___,Roles of white women in Tulara Valley - - - ...,0.885818,False
1,Remember San bows arrows. miss said that? Negr...,65,65_Environmental impacts of eating meat___,Environmental impacts of eating meat - - - ...,1.000000,False
2,unable post!! must :( 2508,106,106_Live streaming and music streaming___,Live streaming and music streaming - - - - ...,0.985714,False
3,"Seems me, white male, fact real hope sort WN s...",-1,-1_useless information___,useless information - - - - - - - - -,0.000000,False
4,"Ironically, presence indoctrinated beta males ...",146,146_Environmental impacts of eating meat___,Environmental impacts of eating meat - - - ...,1.000000,False
...,...,...,...,...,...,...
9995,really programmes Finland???:mad: believed ext...,22,22_Linguistic impact of eating meat___,Linguistic impact of eating meat - - - - -...,1.000000,False
9996,think one real nationalist Finn pay TV-licence...,22,22_Linguistic impact of eating meat___,Linguistic impact of eating meat - - - - -...,1.000000,False
9997,would easy change. problems are. Trouble peopl...,-1,-1_useless information___,useless information - - - - - - - - -,0.000000,False
9998,cool thing Finland law assaulting someone rega...,22,22_Linguistic impact of eating meat___,Linguistic impact of eating meat - - - - -...,1.000000,False


In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy(width=1200)

In [ ]:
topic_model.visualize_barchart()#(top_n_topics=40, n_words=5, height=1200)
# ? topic_model.visualize_barchart

In [ ]:
# more cool built-in methods are available here: https://maartengr.github.io/BERTopic/index.html#overview

In [ ]:
df.head()

,date_posted,title,date,text,topic,cluster_keywords,x,y
0,2011-06-08T14:22:00Z,Re: MOST PEOPLE dont Date or marry INTERRACIALY!,2011-06-08,"Well, must say I'm pleasantly surprised low pe...",9,8_Chinese borrowings of ideas__,13.380741,-3.644559
1,2009-09-06T07:03:00Z,Re: What Africa does not prove,2009-09-06,Remember San bows arrows. miss said that? Negr...,65,64_Environmental impacts of eating meat__,11.179468,-6.588559
2,2005-05-16T18:20:00Z,Help I am unable to post,2005-05-16,unable post!! must :( 2508,106,105_WHITE AMERICANS in Europe__,8.115180,0.109455
3,2008-08-28T15:05:00Z,Re: WN is fantasy at best,2008-08-28,"Seems me, white male, fact real hope sort WN s...",-1,158_Health care in certain areas__,10.443359,-3.096949
4,2008-09-02T13:22:00Z,Re: WN is fantasy at best,2008-09-02,"Ironically, presence indoctrinated beta males ...",146,145_Environmental impacts of playing video gam...,10.682758,-3.505349


In [ ]:
# Topics over time
topics_over_time = topic_model.topics_over_time(df['text'],
                                                df['date'],
                                                global_tuning=False,
                                                evolution_tuning=True,
                                                nr_bins=13)

# Take a look at the data
topics_over_time.head()

,Topic,Words,Frequency,Timestamp
0,-1,"Environmental impacts of eating meat, , , ,",101,2001-09-05 22:22:04.800
1,0,"Environmental impacts of eating meat, , , ,",9,2001-09-05 22:22:04.800
2,1,"Coal emissions and environmental justice, , , ,",4,2001-09-05 22:22:04.800
3,2,"|, , , ,",4,2001-09-05 22:22:04.800
4,4,"environmental impacts of eating meat, , , ,",5,2001-09-05 22:22:04.800


In [ ]:
topics_over_time

,Topic,Words,Frequency,Timestamp,Name
0,-1,"Environmental impacts of eating meat, , , ,",101,2001-09-05 22:22:04.800000000,-1_useless information___
1,0,"Environmental impacts of eating meat, , , ,",9,2001-09-05 22:22:04.800000000,0_ledgering___
2,1,"Coal emissions and environmental justice, , , ,",4,2001-09-05 22:22:04.800000000,1_Zionism___
3,2,"|, , , ,",4,2001-09-05 22:22:04.800000000,2_gun control___
4,4,"environmental impacts of eating meat, , , ,",5,2001-09-05 22:22:04.800000000,4_PG-13 ratings___
...,...,...,...,...,...
825,36,"Shipping and delivery, , , ,",3,2019-07-25 07:23:04.615384576,36_US military___
826,45,"Shipping and delivery issues, , , ,",1,2019-07-25 07:23:04.615384576,45_mods___
827,106,Technology differences between technological a...,1,2019-07-25 07:23:04.615384576,106_Live streaming and music streaming__...
828,138,"Memory management and meat, , , ,",2,2019-07-25 07:23:04.615384576,138_Problems with web browsing___


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, topics=[i for i in range(1,11)])